# Переменные

Ключевое слово **var** создаёт переменную определённого типа, присваивает ей имя и инициализирует её. В общем виде объявление переменной выглядит так:  

var *name* type = *expression*

Либо "type" либо " = expression" можно пропустить, но не оба сразу. Если "type" пропущен, то тип выводится из выражения-инициализатора. Если пропускается " = expression", то переменной присваивается **нулевое значение**(zero value) соответствующего типа. Для чисел он равен __0__, **false** для булевого типа, **""** для строк и **nil** для интерфейсных и ссылочных типов(слайсы, указатели, словари, каналы, функции). Для агрегированных типов каждый элемент принимает нулевое значение. Механизм нулевых значений гарантирует инициализацию переменных известным значением, т.е. в Go отсутствует такое понятие, как неинициализированная переменная. Например, следующий когда выводит пустую строку, а не какой-либо мусор:

In [1]:
import "fmt"
var s string
fmt.Println(s)
fmt.Sprint(s == "")


true


Можно также объявить и инициализировать переменные одной строкой:

In [2]:
// Объявление переменных одного и того же типа.
var a, b, c int

// Объявление переменных разных типов.
var num, flag, name = 4, true, "John"

## Короткое объявление переменных

**Внутри функций** переменные можно также объявить, используя так называемый **метод короткого объявления переменных**(short variable declaration). Общий вид у него такой:

*name* := *expression*

Тип переменной name определяется из выражения expression. Пример:

In [3]:
tree := "oak"
height := 15.7

Наиболее часто именно таким образом и объявляют переменные, оставив объявления var для случая, когда переменная должна иметь определённый тип, который отличается от выводимого из выражения-инициализатора или для случая, когда значение переменной будет присвоено позже и её начальное значение нас не интересует.  
Как и с объявлением var можно сразу **объявить несколько переменных**:

In [4]:
country, population := "Japan", 142

Если при коротком объявлении переменных какие-либо переменные уже были созданы, то им будет просто присвоено новое значение:

In [5]:
day := 1
fmt.Printf("&day = %p\n", &day) // %p выводит адрес.

day, month := 2, "January"
fmt.Sprintf("&day = %p", &day)

&day = 0x7f2b0c5b4208
&day = 0x7f2b0c5b4208


Также при коротком объявление переменных слева от ":=" должна объявляться **хотя бы одна новая** переменная, т.к. иначе будет ошибка компиляции:

In [6]:
planet := "Earth"
radius := 6371

// Ошибка. Так делать нельзя.
// planet, radius := "Mercury", 2440

// Но можно так.
planet, radius, habitable := "Venus", 6052, false

## Указатели

Указатель это переменная, которая хранит адрес переменной. С помощью указателя можно менять значение переменной косвенно(indirectly), т.е. не зная имя переменной. Работа указателей в Go очень похожа работу указателей в C: адрес переменной можно получить с помощью оператора __&__, а получить доступ к значению с помощью оператора разыменовывания(indirection)*****:

In [7]:
suit := "diamond"
ptrSuit := &suit
fmt.Println(*ptrSuit)

*ptrSuit = "hearts"
fmt.Sprint(suit)

diamond
hearts


Каждый компонент агрегированных типов данных, как элемент массива или поле структуры, также является переменной, а следовательно **имеет** и адрес.  
Нулевым значением для указателя любого типа является **nil**. Указатель не равен nil, только если он указывает на какую-либо переменную. Указатели можно сравнивать между собой. Два указателя равны только в случае, когда они указывают на одну и ту же переменную или оба равны nil:

In [8]:
var a, b int
fmt.Sprint(&a == &a, &a == &b, &a == nil)

true false false


В Go возвращение адреса локальной переменной, а затем дальнейшее обращение к этой переменной через указатель **не является ошибкой**. Сборщик мусора не уничтожит переменную, т.к. на неё указывает другая:

In [9]:
func notLucky() *int {
    value := 13
    return &value
}

ptr := notLucky()
fmt.Println(*ptr)

*ptr = 7
fmt.Sprint(*ptr)

13
7


Использование указателей позволяет изменить значение переменной, переданной в функцию:

In [10]:
func luckylize(ptr *int) {
    *ptr = 7
}

guess := 13
luckylize(&guess)
fmt.Sprint(guess)

7


## Встроенная(built-in) функция new

Встроенная функции new служит для выделения памяти. Выражение new(T) создает неименованную(unnamed) переменную типа T, инициализирует её нулевым значением этого типа и возвращает её адрес.

In [11]:
ptrID := new(int)
fmt.Println(*ptrID)

*ptrID = 63
fmt.Sprint(*ptrID)

0
63


Функция new используется довольно редко, т.к. наиболее часто создаваемые неименованные переменные являются структурами, а для них существует более удобный синтаксис литералов.  
Т.к. new является предопределённой функцией, а не ключевым словом, мы можем создать и переменную с таким именем:

In [12]:
// Внутри функции diff переменная new перекрывает предопределённую функцию new.
func diff(old, new int) int {
    return new - old
}

## Время жизни(lifetime) переменных

Время жизни переменной - это интервал существования переменной во время выполнения программы. Переменные уровня пакета существуют до конца выполнения программы. Локальные переменные функций существуют пока они становятся **недостижимыми**(unreachable). Аргументы функции также являются локальными переменными. Они создаются каждый раз, когда вызывается содержащая их функция. В отличие от C в Go локальная переменная необъязательно создаётся в стеке. Рассмотрим следующий пример:

In [13]:
var global *int

func f() {
    var x int
    x = 1
    global = &x
}

func g() {
    y := new(int)
    *y = 1
}

Локальная переменная x внутри функции f должна быть создана в куче, т.к. после завершения выполнения функции, на наё всё ещё ссылается глобальная переменная global. Внутри функции g переменную y можно создать в стеке даже несмотря на то, что она создаётся используя встроенную функцию new. Эта переменная не выходит за рамки функции. Программист не может управлять тем, где именно будет выделена память для создаваемых им переменных. Но это и не нужно делать, т.к. сборщик мусора(garbage collector, или GC) на то и придуман, чтобы освободить разработчика от многих проблем работы с памятью.